In [ ]:
!pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from datasets import load_dataset
import torch
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, ViTFeatureExtractor, ViTForImageClassification
from transformers import AutoModelForTokenClassification
import torch.nn as nn
import os
import json
import torch.nn.functional as F
from pathlib import Path
from shutil import copyfile
from typing import List, Dict, Union, Callable, Optional
from copy import deepcopy

import numpy as np
from scipy.spatial.transform import Rotation as R

import matplotlib as plt

import numpy as np

import os

import pdb
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tqdm import tqdm

In [ ]:
train_path = "/content/train_required_scenes_unified.pt"
labels_path = "/content/labels_required_scenes_unified.pt"

train = torch.load(train_path)
labels = torch.load(labels_path)

In [ ]:
train.size()

torch.Size([1778, 4, 40, 30])

# Finding A-prior probability given lidar intensity

In [ ]:
def merge_close_intensities(intensity_dict, threshold=1.0):
    # Sort the keys for sequential processing
    sorted_keys = sorted(intensity_dict.keys())

    # Initialize the merged dictionary
    merged = {}

    # Process the first key
    if sorted_keys:
        current_key = sorted_keys[0]
        current_prob = intensity_dict[current_key]

        # Process remaining keys
        for key in sorted_keys[1:]:
            # If keys are close, merge them
            if key - current_key < threshold:
                # Update probability (sum them)
                current_prob += intensity_dict[key]
            else:
                # Store the accumulated value and move to next group
                merged[current_key] = current_prob
                current_key = key
                current_prob = intensity_dict[key]

        # Don't forget to add the last group
        merged[current_key] = current_prob

    return merged

# Example usage:
# merged_dict = merge_close_intensities(intensity_prob_dict, threshold=1.0)

def calc_prior_probs(train, labels, pos = True):
    obj_cases = []
    lidar_tensor = train[:, 0, : :].numpy()
    num_objects = labels.size()[1]
    for i in range(num_objects):
        d ={}
        obj_label = labels[:,i,:,:].numpy()
        if pos is True:
            object_present_mask = (obj_label > 0.5)
        else:
            object_present_mask = (obj_label < 0.5)
        lidar_values_where_object = lidar_tensor[object_present_mask]
        lidar_values_where_object = np.round(lidar_values_where_object * 10) / 10
        unique_values, counts = np.unique(lidar_values_where_object, return_counts=True)
        obj_cases.append([unique_values, counts])
        # denumenator = np.sum(counts)
        # intensity_count_dict = {intensity: count/denumenator for intensity, count in zip(unique_values, counts)}
        # intensity_count_dict = merge_close_intensities(intensity_count_dict)
    return obj_cases

def discretize_and_calculate_likelihood(pos_cases, neg_cases, resolution=0.5, debug=True):
    """
    Discretize intensity values to the specified resolution, combine positive and negative cases,
    and calculate likelihood ratio pos_cases / (pos_cases + neg_cases)

    Parameters:
    - pos_cases: List of [intensities, counts] for positive cases
    - neg_cases: List of [intensities, counts] for negative cases
    - resolution: Resolution to discretize the intensity values (default: 0.5)
    - debug: Print debug information

    Returns:
    - Dictionary with structure {intensity: likelihood}
    """
    if debug:
        print(f"Number of positive case objects: {len(pos_cases)}")
        print(f"Number of negative case objects: {len(neg_cases)}")

        # Check first object of each type
        if pos_cases:
            print(f"First positive case shape: Intensities={len(pos_cases[0][0])}, Counts={len(pos_cases[0][1])}")
            if len(pos_cases[0][0]) > 0:
                print(f"Sample intensity range: {min(pos_cases[0][0])} to {max(pos_cases[0][0])}")

        if neg_cases:
            print(f"First negative case shape: Intensities={len(neg_cases[0][0])}, Counts={len(neg_cases[0][1])}")
            if len(neg_cases[0][0]) > 0:
                print(f"Sample intensity range: {min(neg_cases[0][0])} to {max(neg_cases[0][0])}")

    # First get the combined dictionary
    combined_dict = {}
    pos_total = 0
    neg_total = 0

    # Process positive cases
    for i, obj_case in enumerate(pos_cases):
        if len(obj_case) != 2:
            if debug:
                print(f"Warning: Skipping positive case {i} with unexpected format: {obj_case}")
            continue

        intensities, counts = obj_case

        if len(intensities) == 0 or len(counts) == 0:
            if debug:
                print(f"Warning: Skipping empty positive case {i}")
            continue

        if len(intensities) != len(counts):
            if debug:
                print(f"Warning: Mismatch in positive case {i}: {len(intensities)} intensities but {len(counts)} counts")
            continue

        for intensity, count in zip(intensities, counts):
            try:
                # Discretize to the nearest multiple of resolution
                discretized = round(intensity / resolution) * resolution

                # Initialize if not already in the dictionary
                if discretized not in combined_dict:
                    combined_dict[discretized] = [0, 0]

                # Add the count to the positive case (index 0)
                combined_dict[discretized][0] += count
                pos_total += count
            except Exception as e:
                if debug:
                    print(f"Error processing positive intensity {intensity}, count {count}: {e}")

    # Process negative cases
    for i, obj_case in enumerate(neg_cases):
        if len(obj_case) != 2:
            if debug:
                print(f"Warning: Skipping negative case {i} with unexpected format: {obj_case}")
            continue

        intensities, counts = obj_case

        if len(intensities) == 0 or len(counts) == 0:
            if debug:
                print(f"Warning: Skipping empty negative case {i}")
            continue

        if len(intensities) != len(counts):
            if debug:
                print(f"Warning: Mismatch in negative case {i}: {len(intensities)} intensities but {len(counts)} counts")
            continue

        for intensity, count in zip(intensities, counts):
            try:
                # Discretize to the nearest multiple of resolution
                discretized = round(intensity / resolution) * resolution

                # Initialize if not already in the dictionary
                if discretized not in combined_dict:
                    combined_dict[discretized] = [0, 0]

                # Add the count to the negative case (index 1)
                combined_dict[discretized][1] += count
                neg_total += count
            except Exception as e:
                if debug:
                    print(f"Error processing negative intensity {intensity}, count {count}: {e}")

    if debug:
        print(f"Total positive counts: {pos_total}")
        print(f"Total negative counts: {neg_total}")
        print(f"Number of unique discretized intensities: {len(combined_dict)}")
        if combined_dict:
            print(f"Sample of combined dictionary:")
            for i, (k, v) in enumerate(combined_dict.items()):
                print(f"  {k}: {v}")
                if i >= 5:
                    print("  ...")
                    break

    # Calculate the likelihood ratio for each intensity
    likelihood_dict = {}
    for intensity, counts in combined_dict.items():
        pos_count = counts[0]
        neg_count = counts[1]
        total = pos_count + neg_count

        # Calculate the likelihood ratio (avoid division by zero)
        if total > 0:
            likelihood = pos_count / total
            likelihood_dict[intensity] = likelihood

    if debug:
        print(f"Number of intensities with likelihood calculated: {len(likelihood_dict)}")
        if likelihood_dict:
            print(f"Sample of likelihood dictionary:")
            for i, (k, v) in enumerate(sorted(likelihood_dict.items())):
                print(f"  {k}: {v:.4f}")
                if i >= 5:
                    print("  ...")
                    break

    return likelihood_dict

pos_cases = calc_prior_probs(train, labels)
neg_cases = calc_prior_probs(train, labels, pos= False)
cases_dict = discretize_and_calculate_likelihood(pos_cases, neg_cases)

Number of positive case objects: 3
Number of negative case objects: 3
First positive case shape: Intensities=226, Counts=226
Sample intensity range: 0.0 to 100.0
First negative case shape: Intensities=206, Counts=206
Sample intensity range: 0.0 to 100.0
Total positive counts: 29421
Total negative counts: 6371379
Number of unique discretized intensities: 124
Sample of combined dictionary:
  0.0: [np.int64(364), np.int64(6313628)]
  0.5: [np.int64(203), np.int64(406)]
  1.0: [np.int64(1539), np.int64(3078)]
  1.5: [np.int64(444), np.int64(888)]
  2.0: [np.int64(1531), np.int64(3062)]
  2.5: [np.int64(1089), np.int64(2178)]
  ...
Number of intensities with likelihood calculated: 124
Sample of likelihood dictionary:
  0.0: 0.0001
  0.5: 0.3333
  1.0: 0.3333
  1.5: 0.3333
  2.0: 0.3333
  2.5: 0.3333
  ...


In [ ]:
num_objects = 3
x_size = 40
y_size = 30

In [ ]:
# prompt: for each lidar_data, round the values in each cell of the tensor, and get the value from cases_dict and assign it rather than the original intensity value

def replace_intensity_with_likelihood(tensor_data, cases_dict):
  """Replaces intensity values in lidar_data with likelihoods from cases_dict.

  Args:
    lidar_data: A tensor representing lidar data.
    cases_dict: A dictionary mapping discretized intensity values to likelihoods.

  Returns:
    A tensor with intensity values replaced by likelihoods.
  """
  lidar_data = tensor_data[0, :, :]
  # Ensure cases_dict is not empty
  if not cases_dict:
    print("Warning: cases_dict is empty. Returning original lidar_data.")
    return lidar_data

  # Find the minimum and maximum keys in cases_dict for clamping
  min_key = min(cases_dict.keys())
  max_key = max(cases_dict.keys())

  # Iterate over the lidar data and replace intensity values with likelihoods
  new_lidar_data = lidar_data.clone()  # Create a copy to avoid modifying the original tensor
  for i in range(lidar_data.shape[0]):
    for j in range(lidar_data.shape[1]):
      intensity = lidar_data[i, j].item()
      discretized_intensity = round(intensity / 0.5) * 0.5 # Use the same discretization as in the likelihood calculation
      discretized_intensity = max(min(discretized_intensity, max_key), min_key) # Clamp the discretized intensity
      if discretized_intensity in cases_dict:
        new_lidar_data[i, j] = cases_dict[discretized_intensity]
      else:
        # Handle cases where the discretized intensity is not in the dictionary
        # You can choose to either use a default value or the original intensity
        new_lidar_data[i, j] = 0.0  # or lidar_data[i, j, k]

  return new_lidar_data.numpy()

def process_tensor(tensor):
    """
    Processes a tensor based on the provided criteria.

    Args:
      tensor: The input tensor.

    Returns:
      A list of tensors.
    """
    result = []
    for i in range(1, 4):  # Ignore element 0, process elements 1, 2, 3
    #devide by 1000 each element since we multiplied by 100 for enabeling to use Llama model
      result.append((tensor[i, :, :].numpy()) / 1000)
    return result

def opinion_independent_pool_calc(lidar_prob_map, processed_tensor):
  probability_maps = []
  for obj_map in processed_tensor:
    p = np.zeros(lidar_prob_map.shape)
    for i in range(lidar_prob_map.shape[0]):
      for j in range(lidar_prob_map.shape[1]):
        p[i][j] = (lidar_prob_map[i][j] * obj_map[i][j]) / (lidar_prob_map[i][j] * obj_map[i][j] + (1 - lidar_prob_map[i][j]) * (1 - obj_map[i][j]))
        if p[i][j] < 0:
          display("got a negative value!")
          display("p_i_j: ", p[i][j])
          display("lidar: ", lidar_prob_map[i][j])
          display("optical: ", obj_map[i][j])
    probability_maps.append(p)
  return probability_maps




# Calculate Opinion Independent Pool

In [ ]:
# Example usage assuming 'train' is your 440x4x40x30 tensor
from torch.utils.data import TensorDataset, DataLoader, random_split
# Define the split ratio (e.g., 80% train, 20% test)
train_size = int(0.8 * len(train))
test_size = len(train) - train_size

generator = torch.Generator().manual_seed(42)
torch.manual_seed(42)

# Random split
train_dataset, test_dataset = random_split(train, [train_size, test_size], generator = generator)

# Get indices used in the split
train_indices = train_dataset.indices
test_indices = test_dataset.indices

# Split the labels tensor using the same indices
train_labels = labels[train_indices]
test_labels = labels[test_indices]

In [ ]:
test_dataset[0].size()

torch.Size([4, 40, 30])

In [ ]:

processed_tensors = []
prob_tensor = torch.empty(0,num_objects,x_size,y_size)
preds_tensor = torch.empty(0,num_objects,x_size,y_size)
for i in tqdm(range(len(test_dataset))):
    tensor_to_process = test_dataset[i]
    optical_tensors = process_tensor(tensor_to_process)
    lidar_prob_map = replace_intensity_with_likelihood(tensor_to_process, cases_dict)
    probability_maps = opinion_independent_pool_calc(lidar_prob_map, optical_tensors)
    preds = []
    prob_maps = []
    for probability_map in probability_maps:
      rounded_map = np.where(probability_map > 0.5, 1, 0)

      prob_maps.append(probability_map)
      preds.append(rounded_map)

    preds_tensor = torch.cat((preds_tensor, torch.tensor(preds).unsqueeze(0)), dim=0)
    prob_tensor = torch.cat((prob_tensor, torch.tensor(prob_maps).unsqueeze(0)), dim=0)
# prompt: convert labels tensor o dtype-double
labels = labels.double()
prob_tensor = prob_tensor.double()

  0%|          | 0/356 [00:00<?, ?it/s]<ipython-input-10-2210575dd93c>:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  preds_tensor = torch.cat((preds_tensor, torch.tensor(preds).unsqueeze(0)), dim=0)
100%|██████████| 356/356 [00:34<00:00, 10.43it/s]


In [ ]:
class WeightedBCELoss(nn.BCELoss):
    """
    Weighted Binary Cross Entropy Loss that applies higher weights to positive examples.
    Inherits from PyTorch's BCELoss for compatibility.

    Args:
        pos_weight: Weight multiplier for positive examples (default: 10.0)
        reduction: Specifies the reduction to apply to the output ('none', 'mean', 'sum')
        weight: A manual rescaling weight given to the loss of each batch element
    """
    def __init__(self, pos_weight=10.0, weight=None, size_average=None, reduce=None, reduction='mean'):
        super(WeightedBCELoss, self).__init__(weight, size_average, reduce, reduction)
        self.pos_weight = pos_weight
        self.reduction = reduction

    def forward(self, input, target):
        """
        Forward pass for weighted BCE loss

        Args:
            input: Predicted probabilities (B, ...)
            target: Ground truth binary values (B, ...)

        Returns:
            Weighted BCE loss
        """
        # Create weight tensor where positive examples get higher weight
        weights = torch.ones_like(target)
        weights[target == 1] = self.pos_weight

        # Calculate BCE loss element-wise (without reduction)
        bce = F.binary_cross_entropy(input, target, reduction='none')

        # Apply weights to the loss
        weighted_bce = bce * weights

        # Apply reduction
        if self.reduction == 'mean':
            return weighted_bce.mean()
        elif self.reduction == 'sum':
            return weighted_bce.sum()
        else:  # 'none'
            return weighted_bce

In [ ]:
prob_tensor = prob_tensor.double()
test_labels = test_labels.double()

In [ ]:
# prompt: calculate the loss between prob_tensor and label_batch. use BCELoss
pos_weight = torch.tensor([10.0])
criterion = WeightedBCELoss(pos_weight=pos_weight)
loss = criterion(prob_tensor, test_labels)
loss

tensor(4.2891, dtype=torch.float64)

In [ ]:
# prompt: calculate accuracy score, precision, recall and f1 score

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Flatten the predictions and labels for easier calculations
preds_flat = preds_tensor.flatten().numpy()
labels_flat = test_labels.flatten().numpy()

# Calculate metrics
accuracy = accuracy_score(labels_flat, preds_flat)
precision = precision_score(labels_flat, preds_flat)
recall = recall_score(labels_flat, preds_flat)
f1 = f1_score(labels_flat, preds_flat)
auc = roc_auc_score(labels_flat, preds_flat)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"AUC Score: {auc}")


Accuracy: 0.9953206928838951
Precision: 0.36743515850144093
Recall: 0.04386719421985206
F1 Score: 0.07837713231904103
AUC Score: 0.5217615464462965
